In [1]:
import cv2
from matplotlib.pyplot import imshow
import torch
import numpy as np
import time
import torchvision.transforms as transforms
from PIL import Image

In [ ]:
def preprocess_image(image):
        image = Image.fromarray(image)
        normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
         )
        preprocess = transforms.Compose([
            transforms.Resize((480, 640)),
            transforms.ToTensor(),
            normalize
        ])
        
        return preprocess(image).unsqueeze(0)

In [ ]:
def test():
    model = torch.hub.load('hustvl/yolop', 'yolop', pretrained=True)
    vid = cv2.VideoCapture(0) 
    
    while(True): 
        start_time = time.time()
        
        ret, frame = vid.read() 
        frame = cv2.resize(frame, dsize=(640, 480))
        frame = np.asanyarray(frame)
        img = preprocess_image(frame)
        det_out, da_seg_out,ll_seg_out = model(img)
        
        cv2.imshow('rgb', frame) 
        end_time = time.time()
        print(1/(end_time-start_time))

        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break
    
    vid.release() 
    cv2.destroyAllWindows() 

In [ ]:
test()

In [ ]:
import torch.onnx
import onnx
import tensorrt as trt

# Определите модель нейронной сети
class YourModel(torch.nn.Module):
    def __init__(self):
        super(YourModel, self).__init__()
        # Здесь определение слоев вашей нейронной сети

# Создайте экземпляр модели
model = YourModel()

# Загрузите параметры модели из файла .pth
model.load_state_dict(torch.load('путь_к_файлу.pth', map_location=torch.device('cpu')))
model.eval()

# Экспорт модели в формат ONNX
dummy_input = torch.randn(1, 3, 224, 224)  # Пример входных данных
onnx_file_path = 'model.onnx'
torch.onnx.export(model, dummy_input, onnx_file_path, verbose=True)




In [2]:
import torch.onnx
import onnx
import tensorrt as trt
import os
# Компиляция ONNX в TensorRT
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
with trt.Builder(TRT_LOGGER) as builder, builder.create_network() as network, trt.OnnxParser(network, TRT_LOGGER) as parser:
    builder.max_workspace_size = 1 << 30  # 1GB
    builder.max_batch_size = 1
    # Загрузка ONNX модели
    with open('yolop-320-320.onnx', 'rb') as model_file:
        parser.parse(model_file.read())
    # Компиляция модели
    engine = builder.build_cuda_engine(network)

# Сохранение TensorRT модели на диск
trt_file_path = 'yolop-320-320.trt'
with open(trt_file_path, 'wb') as f:
    f.write(engine.serialize())

/home/jetson/.local/lib/python3.8/site-packages/onnx/mapping.py:27: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  int(TensorProto.STRING): np.dtype(np.object)


AttributeError: module 'numpy' has no attribute 'object'.
`np.object` was a deprecated alias for the builtin `object`. To avoid this error in existing code, use `object` by itself. Doing this will not modify any behavior and is safe. 
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations